In [6]:
import requests
import json
import time

API_KEY = "7aa8ce86-cdbb-40f8-8dd9-0eaa64da1a58"

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {API_KEY}"
}

IMAGES = [
    "https://cdn.nba.com/headshots/nba/latest/1040x760/2544.png",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/LeBron_James_-_51959723161_%28cropped%29.jpg/1200px-LeBron_James_-_51959723161_%28cropped%29.jpg",
    "https://phantom-marca.unidadeditorial.es/52ad7861d19d3222a3cb622cf94f4ca7/resize/1320/f/jpg/assets/multimedia/imagenes/2022/09/22/16638070009361.jpg"
]

In [7]:
def create_model(title):
    url = "https://api.tryleap.ai/api/v1/images/models"

    payload = {
        "title": title,
        "subjectKeyword": "@me"
    }

    response = requests.post(url, json=payload, headers=HEADERS)

    model_id = json.loads(response.text)["id"]
    return model_id


def upload_image_samples(model_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/samples/url"

    payload = {"images": IMAGES}
    response = requests.post(url, json=payload, headers=HEADERS)


def queue_training_job(model_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/queue"
    response = requests.post(url, headers=HEADERS)
    data = json.loads(response.text)

    print(response.text)

    version_id = data["id"]
    status = data["status"]

    print(f"Version ID: {version_id}. Status: {status}")

    return version_id, status


def get_model_version(model_id, version_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/versions/{version_id}"
    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    version_id = data["id"]
    status = data["status"]

    print(f"Version ID: {version_id}. Status: {status}")

    return version_id, status


def generate_image(model_id, prompt):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences"

    payload = {
        "prompt": prompt,
        "steps": 50,
        "width": 512,
        "height": 512,
        "numberOfImages": 1,
        "seed": 4523184
    }

    response = requests.post(url, json=payload, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    status = data["status"]

    print(f"Inference ID: {inference_id}. Status: {status}")

    return inference_id, status


def get_inference_job(model_id, inference_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences/{inference_id}"

    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    state = data["state"]
    image = None

    if len(data["images"]):
        image = data["images"][0]["uri"]

    print(f"Inference ID: {inference_id}. State: {state}")

    return inference_id, state, image




In [8]:
# Let's create a custom model so we can fine tune it.
model_id = create_model("Sample")

# We now upload the images to fine tune this model.
upload_image_samples(model_id)

# Now it's time to fine tune the model. Notice how I'm continuously 
# getting the status of the training job and waiting until it's
# finished before moving on.
version_id, status = queue_training_job(model_id)
while status != "finished":
    time.sleep(10)
    version_id, status = get_model_version(model_id, version_id)



{"createdAt":"2023-03-14T16:41:41.957292+00:00","id":"9bf1e7e7-3bd2-403d-9a24-a99a962995d6","status":"queued"}
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a962995d6. Status: queued
Version ID: 9bf1e7e7-3bd2-403d-9a24-a99a9629

In [12]:
# Now that we have a fine-tuned version of a model, we can
# generate images using it. Notice how I'm using '@me' to 
# indicate I want pictures similar to the ones we upload to 
# fine tune this model.
inference_id, status = generate_image(
    model_id, 
    prompt="A photo of @me with sunglasses"
)
while status != "finished":
    time.sleep(10)
    inference_id, status, image = get_inference_job(model_id, inference_id)

print(image)

Inference ID: 0d3402fd-6b49-4d5f-a675-22fb79c6bd58. Status: queued
Inference ID: 0d3402fd-6b49-4d5f-a675-22fb79c6bd58. State: queued
Inference ID: 0d3402fd-6b49-4d5f-a675-22fb79c6bd58. State: finished
https://dreamtrain.s3.us-west-2.amazonaws.com/image-gen-0d3402fd-6b49-4d5f-a675-22fb79c6bd58/generated_images/0.png
